In [1]:
#Problem Setup/Definition:
import numpy as np
np.random.seed(42)
import random
random.seed(42)
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfTransformer
from string import punctuation
from string import digits
from nltk.corpus import wordnet
from sklearn.feature_extraction import text, stop_words
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.svm import LinearSVC
import math 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.externals.joblib import Memory
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD

import pandas as pd
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
def my_custom_preprocessor(doc_string):
    # do all data preprocessing here
    
    # Lower case
    doc_string=doc_string.lower()
    
    # Remove Numbers
    remove_digits = str.maketrans('', '', digits)
    doc_string.translate(remove_digits)
    
    # Convert to tokenized form....
    tokens = nltk.tokenize.word_tokenize(doc_string)
    # Iterate through list of tokens (words) and remove all numbers
    tokens = [word for word in tokens if word.isalpha()]
    # Iterate through list of tokens (words) and stem (shorten) each word
    port_stemmer = PorterStemmer()
    tokens = [port_stemmer.stem(words) for words in tokens ]
    
    ###############################
    #### Lemmatize with pos_tag ###
    ###############################
    
    lemmatizer = WordNetLemmatizer()
    
    # Convert between two different tagging schemes
    def change_tags(penntag):
        morphy_tag = {'NN':'n', 'JJ':'a',
                      'VB':'v', 'RB':'r'}
        try:
            return morphy_tag[penntag[:2]]
        except:
            return 'n'
        
    tokens = [lemmatizer.lemmatize(word.lower(), pos=change_tags(tag)) for word, tag in pos_tag(tokens)]
    
    # Rejoin List of tokens and return that single document-string
    return ' '.join(tokens)

###########################
#### RoC Curve Function ###
###########################

def plot_roc(fpr, tpr):
    fig, ax = plt.subplots()

    roc_auc = auc(fpr,tpr)

    ax.plot(fpr, tpr, lw=2, label= 'area under curve = %0.4f' % roc_auc)

    ax.grid(color='0.7', linestyle='--', linewidth=1)

    ax.set_xlim([-0.1, 1.1])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate',fontsize=15)
    ax.set_ylabel('True Positive Rate',fontsize=15)

    ax.legend(loc="lower right")

    for label in ax.get_xticklabels()+ax.get_yticklabels():
        label.set_fontsize(15)

def fit_predict_and_plot_roc(pipe, train_data, train_label, test_data, test_label):
    pipe.fit(train_data, train_label)

    if hasattr(pipe, 'decision_function'):
        prob_score = pipe.decision_function(test_data)
        fpr, tpr, _ = roc_curve(test_label, prob_score)
    else:
        prob_score = pipe.predict_proba(test_data)
        fpr, tpr, _ = roc_curve(test_label, prob_score[:,1])

    plot_roc(fpr, tpr)
    
#####################################################
#### Define Custom stop words for CountVectorizer ###
#####################################################

stop_words_skt = text.ENGLISH_STOP_WORDS
stop_words_en = stopwords.words('english')
combined_stopwords = set.union(set(stop_words_en),set(punctuation),set(stop_words_skt))

# Run stop_words through the same pre-processor as the document-matrix
# This will apply stemmed/lemmatized stop_woirds to stemmed/lemmatized tokenized document lists
def process_stop_words(stop_word_set):
    doc_string = ' '.join(stop_word_set)
    return my_custom_preprocessor(doc_string).split()

################################
#### Estimator Helper Class  ###
################################

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

##################################
#### Import Dataset Train/Test ###
##################################

# Only take a specific selection (8) of the 20 available categories
categories = ['comp.graphics', 'comp.os.ms-windows.misc',
'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
'rec.autos', 'rec.motorcycles',
'rec.sport.baseball', 'rec.sport.hockey']

# Load a training & test data sets consisting of those 8 categories
train_dataset = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = None)
test_dataset = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = None)

## Load training & test data sets WITHOUT headers & footers
train_dataset_no_hf = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = None, remove=['headers', 'footers'])
test_dataset_no_hf = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = None, remove=['headers', 'footers'])

print("\n\n" + '-'*40 + "\n\n")

#############################################
#### Define Class data set arrys (0 or 1) ###
#############################################
# Categorize the 8 news categories into two (binary) Classes 
# 0 = computer technology
# 1 = recreational activity
training_data_class = [] 
test_data_class = []

# Categories are mapped 0-7, (0-3) = Comp, (4-7) = Recreation
print(train_dataset.target_names)
print(train_dataset.target)

for category in train_dataset.target:
    if category < 4:
        training_data_class.append(0)
    else:
        training_data_class.append(1)
        
# Reshape test dataset
for category in test_dataset.target:
    if category < 4:
        test_data_class.append(0)
    else:
        test_data_class.append(1)
        
# Sanity Checks, values should all be either 1 or 0
print("First 10 articles Classification (Train): \n" + str(training_data_class[0:10]))
print("First 10 articles Classification (Train): \n" + str(test_data_class[0:10]))
# This will be used for the classification categories only!!!!
# Each data point refers to the classification of a single article in dataset



----------------------------------------


['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']
[6 7 4 ... 6 6 2]
First 10 articles Classification (Train): 
[1, 1, 1, 0, 0, 0, 0, 1, 1, 0]
First 10 articles Classification (Train): 
[1, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [3]:
#####################################
## Create Pipelines for Comparison ##
#####################################
#enable Cachine
cachedir = mkdtemp()
memory = Memory(cachedir=cachedir, verbose=0)

### Initial Pipeline ###
# These tuples() will be altered via the 'param_grid' List[]
pipeline_hf = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('dim_reducer', TruncatedSVD()),
    ('classifier', LinearSVC(max_iter=5000)),
],
memory=memory
)

pipeline_no_hf = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('dim_reducer', TruncatedSVD()),
    ('classifier', LinearSVC(max_iter=5000)),
],
memory=memory
)

######################
## Cross Validation ##
######################
# An List[] of Dictionary{key:value} parameters that will be iterated over
# Each Dictionary{} in the List[] references different types of 'vectorizer', 'tfidf', etc.
# Alter the range of hyperparameters within each Dictionary{} with <estimator>__<parameter>.
# E.g. Try both 3 & 5 min_df values for CountVectorizer().... '<vectorizer>__<min_df>'': [3,5] 

# Options to Iterate Over:

MIN_DIF = [3,5]
# Lemm + Stemm or Defualt
TOKEN_PATTERN = [r'(?u)\b\w\w+\b',r'(?u)\b[^\W\d_][^\W\d_][^\W\d_]+\b']
REDUCER_OPTIONS = [TruncatedSVD(), NMF()]
REDUCER_N_COMPONENTS = [50]
# LinearSVC() values
OPTIMAL_LINEAR_C_VALUE = [100]
# LogisticRegression() Values
LOG_REG_PENALTIES = ['l1', 'l2']
OPTIMAL_LOG_REG_C_VALUE = [100]

param_grid = [
                { # Linear Classifier 
                    'vectorizer__min_df': MIN_DIF,
                    'vectorizer__token_pattern': TOKEN_PATTERN,
                    'dim_reducer': REDUCER_OPTIONS,
                    'dim_reducer__n_components': REDUCER_N_COMPONENTS,
                    'classifier': [LinearSVC()],
                    'classifier__C':OPTIMAL_LINEAR_C_VALUE
                },
    
                { # Logisitc Regresion
                    'vectorizer__min_df': MIN_DIF,
                    'vectorizer__token_pattern': TOKEN_PATTERN,
                    'dim_reducer': REDUCER_OPTIONS,
                    'dim_reducer__n_components': REDUCER_N_COMPONENTS,
                    'classifier': [LogisticRegression(solver='liblinear', max_iter=5000)],
                    'classifier__penalty': LOG_REG_PENALTIES,
                    'classifier__C':OPTIMAL_LOG_REG_C_VALUE                        
                },
    
                { # Naive Bayes Gaussian
                    'vectorizer__min_df': MIN_DIF,
                    'vectorizer__token_pattern': TOKEN_PATTERN,
                    'dim_reducer': REDUCER_OPTIONS,
                    'dim_reducer__n_components': REDUCER_N_COMPONENTS,
                    'classifier': [GaussianNB()],                    
                }    
            ]

/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp/tmpro8tpnpr'", use "location='/tmp/tmpro8tpnpr'" instead.
  


In [4]:
## Cross Validate/iterate over pipeline; data has header/footer included
grid = GridSearchCV(pipeline_hf, cv=5, n_jobs=1, param_grid=param_grid, scoring='accuracy')
grid.fit(train_dataset.data, train_dataset.target)

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 3.21s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 3.15s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 fu

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 2.23s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 2.21s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 fu

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=Memory(location=/tmp/tmpro8tpnpr/joblib),
                                steps=[('vectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                  

In [5]:
## Cross Validate/iterate over pipeline; data has header/footer removed
grid_no_hf = GridSearchCV(pipeline_no_hf, cv=5, n_jobs=1, param_grid=param_grid, scoring='accuracy')
grid_no_hf.fit(train_dataset_no_hf.data, train_dataset.target)

rmtree(cachedir)

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 2.71s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 2.59s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 fu

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 1.79s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 1.82s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 fu

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

In [12]:
#######################################################
## Grid Search Results : DATA with HEADERS + FOOTERS ##
#######################################################
# Add column to Table: Used data with Header and Footers INCLUDED
table_hf = pd.DataFrame(grid.cv_results_)
table_hf.insert (len(table_hf.columns)-1, 'Has Header + Footer', 'True')

# Print and order by best 'accuracy'
table_hf.sort_values(by=['rank_test_score'])

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
10       1.408059      0.090800         0.193447        0.004733   
8        1.484378      0.033207         0.201690        0.004794   
16       0.871167      0.005852         0.201438        0.007013   
2        6.498781      0.081996         0.199659        0.003200   
0        7.954103      0.129723         0.202872        0.007657   
18       0.798886      0.007357         0.195079        0.004098   
9        1.189883      0.040266         0.187449        0.002881   
17       0.697118      0.008917         0.189007        0.003977   
11       1.209839      0.032485         0.184384        0.004553   
1        6.429940      0.061321         0.188422        0.005113   
19       0.646897      0.011224         0.185034        0.004562   
3        5.531213      0.121537         0.187000        0.005481   
13       0.614347      0.018574         0.273947        0.004654   
12       0.798599      0.027466         0.306092        0.009427   
14       0.727121      0.023210         0.283147        0.009917   
15       0.575513      0.016831         0.259748        0.002222   
5       21.070481      4.272459         0.279576        0.007013   
4       26.462240      5.118694         0.313854        0.014846   
7       14.989134      3.312757         0.261963        0.003023   
6       15.916521      3.832389         0.284388        0.009830   
21       0.502825      0.005262         0.276925        0.003266   
22       0.592215      0.009808         0.283581        0.007789   
23       0.447910      0.007014         0.259050        0.004108   
20       0.663283      0.007550         0.310310        0.010301   
26       0.465228      0.005912         0.197505        0.005717   
24       0.541731      0.006244         0.200918        0.006454   
27       0.313995      0.003311         0.187235        0.003776   
29       0.376638      0.003288         0.281044        0.006284   
25       0.373937      0.003874         0.190351        0.003808   
31       0.320980      0.007667         0.264104        0.004610   
30       0.471277      0.008275         0.287955        0.011295   
28       0.548457      0.010872         0.312255        0.010124   

                                     param_classifier param_classifier__C  \
10  LogisticRegression(C=100, class_weight=None, d...                 100   
8   LogisticRegression(C=100, class_weight=None, d...                 100   
16  LogisticRegression(C=100, class_weight=None, d...                 100   
2   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
0   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
18  LogisticRegression(C=100, class_weight=None, d...                 100   
9   LogisticRegression(C=100, class_weight=None, d...                 100   
17  LogisticRegression(C=100, class_weight=None, d...                 100   
11  LogisticRegression(C=100, class_weight=None, d...                 100   
1   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
19  LogisticRegression(C=100, class_weight=None, d...                 100   
3   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
13  LogisticRegression(C=100, class_weight=None, d...                 100   
12  LogisticRegression(C=100, class_weight=None, d...                 100   
14  LogisticRegression(C=100, class_weight=None, d...                 100   
15  LogisticRegression(C=100, class_weight=None, d...                 100   
5   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
4   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
7   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
6   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
21  LogisticRegression(C=100, class_weight=None, d...                 100   
22  LogisticRegression(C=100, class_weight=None, d...            

In [13]:
#######################################################
## Grid Search Results : DATA with HEADERS + FOOTERS ##
#######################################################
# Add column to Table: Used data with Header and Footers REMOVED
table_no_hf = pd.DataFrame(grid_no_hf.cv_results_)
table_no_hf.insert (len(table_no_hf.columns)-1, 'Has Header + Footer', 'Fasle')

# Print and order by best 'accuracy'
table_no_hf.sort_values(by=['rank_test_score'])


mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
23       0.378391      0.006370         0.202152        0.007484   
16       0.670012      0.020358         0.161506        0.007492   
8        0.886474      0.024418         0.159901        0.008201   
15       0.455323      0.010623         0.207047        0.007051   
10       0.851201      0.017299         0.156837        0.008664   
7       14.365302      1.582189         0.205007        0.007871   
18       0.592844      0.014176         0.155059        0.008051   
14       0.567618      0.016777         0.218685        0.009512   
0       11.582217      0.060959         0.163484        0.005727   
1       10.139840      0.108874         0.148912        0.007119   
2       10.079398      0.143874         0.157957        0.010793   
4       18.060695      4.602733         0.243078        0.008499   
22       0.495753      0.017182         0.219551        0.010009   
24       0.464147      0.016501         0.164031        0.010452   
25       0.322362      0.007512         0.152286        0.006579   
9        0.735852      0.007769         0.148799        0.006980   
17       0.517650      0.008764         0.148489        0.006593   
12       0.631402      0.017905         0.239947        0.007539   
29       0.333229      0.006752         0.228689        0.007334   
27       0.269844      0.003732         0.147499        0.006766   
20       0.563216      0.020155         0.239634        0.007226   
30       0.398117      0.017051         0.220420        0.009587   
5       17.172029      4.300595         0.217781        0.007423   
6       16.374611      2.703669         0.221115        0.008801   
28       0.467424      0.019192         0.247353        0.010335   
13       0.498140      0.014438         0.213683        0.007533   
3        9.331931      0.123331         0.148830        0.011985   
31       0.276968      0.006380         0.208154        0.008411   
19       0.467830      0.007138         0.147230        0.006690   
11       0.721603      0.038945         0.149164        0.008442   
26       0.395729      0.019597         0.158933        0.007624   
21       0.425403      0.008214         0.215795        0.008208   

                                     param_classifier param_classifier__C  \
23  LogisticRegression(C=100, class_weight=None, d...                 100   
16  LogisticRegression(C=100, class_weight=None, d...                 100   
8   LogisticRegression(C=100, class_weight=None, d...                 100   
15  LogisticRegression(C=100, class_weight=None, d...                 100   
10  LogisticRegression(C=100, class_weight=None, d...                 100   
7   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
18  LogisticRegression(C=100, class_weight=None, d...                 100   
14  LogisticRegression(C=100, class_weight=None, d...                 100   
0   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
1   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
2   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
4   LinearSVC(C=1.0, class_weight=None, dual=True,...                 100   
22  LogisticRegression(C=100, class_weight=None, d...                 100   
24       GaussianNB(priors=None, var_smoothing=1e-09)                 NaN   
25       GaussianNB(priors=None, var_smoothing=1e-09)                 NaN   
9   LogisticRegression(C=100, class_weight=None, d...                 100   
17  LogisticRegression(C=100, class_weight=None, d...                 100   
12  LogisticRegression(C=100, class_weight=None, d...                 100   
29       GaussianNB(priors=None, var_smoothing=1e-09)                 NaN   
27       GaussianNB(priors=None, var_smoothing=1e-09)                 NaN   
20  LogisticRegression(C=100, class_weight=None, d...                 100   
30       GaussianNB(priors=None, var_smoothing=1e-09)            

In [25]:
#########################################################################
## Combine Findings of both Tables (W/wo) HEADERS + FOOTERS IN DATASET ##
#########################################################################

# Combine Both Tables
combined_table = pd.concat([table_no_hf, table_hf])
combined_table.sort_values(by=['rank_test_score'])

## Note: Still some weirdness when combining tables. Overlapping indexes for ranking
## Using data without headers/foots results in incredibly low scores

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
23       0.378391      0.006370         0.202152        0.007484   
10       1.408059      0.090800         0.193447        0.004733   
16       0.670012      0.020358         0.161506        0.007492   
8        1.484378      0.033207         0.201690        0.004794   
16       0.871167      0.005852         0.201438        0.007013   
..            ...           ...              ...             ...   
31       0.320980      0.007667         0.264104        0.004610   
26       0.395729      0.019597         0.158933        0.007624   
30       0.471277      0.008275         0.287955        0.011295   
21       0.425403      0.008214         0.215795        0.008208   
28       0.548457      0.010872         0.312255        0.010124   

                                     param_classifier param_classifier__C  \
23  LogisticRegression(C=100, class_weight=None, d...                 100   
10  LogisticRegression(C=100, class_weight=None, d...                 100   
16  LogisticRegression(C=100, class_weight=None, d...                 100   
8   LogisticRegression(C=100, class_weight=None, d...                 100   
16  LogisticRegression(C=100, class_weight=None, d...                 100   
..                                                ...                 ...   
31       GaussianNB(priors=None, var_smoothing=1e-09)                 NaN   
26       GaussianNB(priors=None, var_smoothing=1e-09)                 NaN   
30       GaussianNB(priors=None, var_smoothing=1e-09)                 NaN   
21  LogisticRegression(C=100, class_weight=None, d...                 100   
28       GaussianNB(priors=None, var_smoothing=1e-09)                 NaN   

                                    param_dim_reducer  \
23  NMF(alpha=0.0, beta_loss='frobenius', init=Non...   
10  TruncatedSVD(algorithm='randomized', n_compone...   
16  TruncatedSVD(algorithm='randomized', n_compone...   
8   TruncatedSVD(algorithm='randomized', n_compone...   
16  TruncatedSVD(algorithm='randomized', n_compone...   
..                                                ...   
31  NMF(alpha=0.0, beta_loss='frobenius', init=Non...   
26  TruncatedSVD(algorithm='randomized', n_compone...   
30  NMF(alpha=0.0, beta_loss='frobenius', init=Non...   
21  NMF(alpha=0.0, beta_loss='frobenius', init=Non...   
28  NMF(alpha=0.0, beta_loss='frobenius', init=Non...   

   param_dim_reducer__n_components param_vectorizer__min_df  \
23                              50                        5   
10                              50                        5   
16                              50                        3   
8                               50                        3   
16                              50                        3   
..                             ...                      ...   
31                              50                        5   
26                              50                        5   
30                              50                        5   
21                              50                        3   
28                              50                        3   

      param_vectorizer__token_pattern  ...  \
23  (?u)\b[^\W\d_][^\W\d_][^\W\d_]+\b  ...   
10                      (?u)\b\w\w+\b  ...   
16                      (?u)\b\w\w+\b  ...   
8                       (?u)\b\w\w+\b  ...   
16                      (?u)\b\w\w+\b  ...   
..                                ...  ...   
31  (?u)\b[^\W\d_][^\W\d_][^\W\d_]+\b  ...   
26                      (?u)\b\w\w+\b  ...   
30                      (?u)\b\w\w+\b  ...   
21  (?u)\b[^\W\d_][^\W\d_][^\W\d_]+\b  ...   
28                      (?u)\b\w\w+\b  ...   

                                               params split0_test_score  \
23  {'classifier': LogisticRegression(C=100, class...          0.142555   
10  {'classifier': LogisticRegression(C=100, class...          0.863780   
16  {'classifier': LogisticRegression(C=10